In [1]:
import re
import numpy as np

In [2]:
with open("/content/drive/MyDrive/made/adv_ml/corpora/WarAndPeace.txt", "r") as f:
    war_and_peace = f.read()

train_text = war_and_peace

**1. Реализуйте базовый частотный метод по Шерлоку Холмсу:**
* **подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);**

In [3]:
def preprocess_text(text):
    prep_text = re.sub("[^а-я ]", " ", text.lower())
    return re.sub(' +', ' ', prep_text)

In [4]:
def count_frequencies(text):
    freqs = {chr(i): 0 for i in range(ord('а'), ord('я') + 1)}
    freqs[' '] = 0
    
    for sym in text:
        freqs[sym] += 1
    return freqs

In [5]:
train_text = preprocess_text(train_text)
train_unigram_freqs = count_frequencies(train_text)

In [6]:
train_unigram_freqs

{' ': 104468,
 'а': 45209,
 'б': 9310,
 'в': 24824,
 'г': 11177,
 'д': 16387,
 'е': 42519,
 'ж': 5460,
 'з': 9602,
 'и': 35838,
 'й': 6210,
 'к': 19328,
 'л': 27277,
 'м': 15940,
 'н': 35119,
 'о': 61282,
 'п': 13847,
 'р': 24570,
 'с': 28128,
 'т': 30619,
 'у': 15454,
 'ф': 1209,
 'х': 4600,
 'ц': 2179,
 'ч': 7349,
 'ш': 5090,
 'щ': 1514,
 'ъ': 283,
 'ы': 10233,
 'ь': 10498,
 'э': 1629,
 'ю': 3495,
 'я': 12477}

* **возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;**

In [7]:
test_text = """Надо больше доверять себе, - сказал он. - Пора бы мне знать, что 
если какой-нибудь факт идет вразрез с длинной цепью логических заключений, 
значит, его можно истолковать иначе. В коробке лежало две пилюли - в одной 
содержался смертельный яд, другая - совершенно безвредная. Как это я не 
догадался раньше, чем увидел коробку"""

test_text = preprocess_text(test_text)

In [8]:
alphabet = sorted(train_unigram_freqs.keys())
rand_alphabet = list(np.random.permutation(alphabet))

In [9]:
def change_alphabet(alphabet_from, alphabet_to, text):
    encoded_text = ""
    for sym in text:
        encoded_text += alphabet_to[alphabet_from.index(sym)]
    return encoded_text

In [10]:
test_text_encoded = change_alphabet(alphabet, rand_alphabet, test_text)

In [11]:
test_text_encoded

'хвряъщяпеалъряюлшчйеътлщлъткв впъяхъняшвъщсъмхлъ хвйеъыйяълтпдъквкяэъхдщуреъгвкйъдрлйъюшв шл ътърпдххяэъолнефъпяидылткдцъ вкпфылхдэъ хвыдйълияъмябхяъдтйяпкяювйеъдхвылъюъкяшящклъплбвпяърюлъндпфпдъюъярхяэътярлшбвптчътмлшйлпехсэъчрършуивчътяюлшалххяъщл юшлрхвчъквкъжйяъчъхлъряиврвптчъшвхеалъылмъуюдрлпъкяшящку'

* **расшифруйте их таким частотным методом**

1. Соберем частоты символов в зашифрованном тексте.
2. Отсортируем списки частот символов в корпусе и зашифрованном тексте.
3. Заменим символы из отсортированного списка символов зашифрованного текста на символы в соответствующих позициях в отсортированном списке символов корпуса.

In [12]:
def sort_alpabet_by_freq(freqs):
    return list(map(lambda x: x[0], sorted(freqs.items(), 
                                           key=lambda x: x[1], 
                                           reverse=True)))

In [13]:
test_unigram_freqs = count_frequencies(test_text_encoded)

sorted_train_alphabet = sort_alpabet_by_freq(train_unigram_freqs)
sorted_test_alphabet = sort_alpabet_by_freq(test_unigram_freqs)

test_text_decoded = change_alphabet(sorted_test_alphabet, sorted_train_alphabet, test_text_encoded)

In [14]:
test_text_decoded

'иета панмчо тадовукм ропо рлеяен аи жаве пх зио яиекм гка орнс лелаь испбтм юелк сток двеявоя р тнсииаь эожмш наысгорлсщ яелншгоись яиегск оыа зайиа срканладекм сиего д лавапло нойена тдо жсншнс д атиаь ратовйенру рзовконмихь ут твбыеу радовчоииа поядвотиеу лел цка у ио таыетенру веимчо гоз бдстон лаваплб'

In [15]:
def calc_accuracy(true_text, pred_text):
    coincide = 0
    for i in range(len(true_text)):
        if true_text[i] == pred_text[i]:
            coincide += 1
    return coincide / len(true_text) * 100

In [16]:
print(f"Доля правильно расшифрованных букв: {round(calc_accuracy(test_text, test_text_decoded), 2)}%")

Доля правильно расшифрованных букв: 15.69%


**2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:**

* **подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;**

In [17]:
def count_bigram_frequencies(text):
    freqs = {}

    all_combinations = [chr(i) for i in range(ord('а'), ord('я') + 1)]
    all_combinations += [' ']

    for sym1 in all_combinations:
        for sym2 in all_combinations:
            freqs[sym1 + sym2] = 0
    
    for i in range(len(text) - 1):
        freqs[text[i] + text[i+1]] += 1
    return freqs

In [18]:
train_bigram_freqs = count_bigram_frequencies(train_text)
test_bigram_freqs = count_bigram_frequencies(test_text_encoded)

sorted_train_bigrams_alphabet = sort_alpabet_by_freq(train_bigram_freqs)
sorted_test_bigrams_alphabet = sort_alpabet_by_freq(test_bigram_freqs)

* **проведите тестирование аналогично п.1, но при помощи биграмм**

1. Для первой биграммы заменим биграмму из отсортированного списка символов зашифрованного текста на биграмму в соответствующей позиции в отсортированном списке символов корпуса.
2. Для последующих биграмм находим стартовую позицию для поиска аналогично п.1. Затем, начиная с этой позиции, ищем ближайшую биграмму (до и после текущей позиции), которая начинается с символа, на который заканчивается текущая строка.

In [19]:
def decode_bigrams(alphabet_from, alphabet_to, text):
    decoded_text = ""
    bigram = text[:2]
    decoded_text += alphabet_to[alphabet_from.index(bigram)]

    for i in range(1, len(text) - 1):
        bigram = text[i] + text[i+1]
        next = prev = alphabet_from.index(bigram)
        
        decoded_next = alphabet_to[next]
        decoded_prev = alphabet_to[prev]
        while decoded_prev[0] != decoded_text[-1] and decoded_next[0] != decoded_text[-1]:
            if prev != 0:
                prev -= 1
            if next != len(alphabet_to) - 1:
                next += 1
            decoded_next = alphabet_to[next]
            decoded_prev = alphabet_to[prev]
        if decoded_prev[0] == decoded_text[-1]:
            decoded_text += decoded_prev[1]
        elif decoded_next[0] == decoded_text[-1]:
            decoded_text += decoded_next[1]

    return decoded_text

In [20]:
bigrams_decoded_text = decode_bigrams(sorted_test_bigrams_alphabet, sorted_train_bigrams_alphabet, test_text_encoded)
bigrams_decoded_text

'е де бем м стол кры о эт сталазан язая бы жеманале де ры ск этал нал ритуди же ал деговерожеля этран ил жегльк мой илобыл у ану м у бы дня рудемадал был зналя и бы де бы олой а руде м ператру эталеморал ск был нил сль желой этей пей м стал кран и бы я чел де и адоводем стодеру м си уде скобокну бы и ть чт'

In [21]:
print(f"Доля правильно расшифрованных букв: {round(calc_accuracy(test_text, bigrams_decoded_text), 2)}%")

Доля правильно расшифрованных букв: 13.4%


**3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:**

* **предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;**

1. Начнем с рандомной перестановки символов
2. На каждой итерации:
* Кандидат в новое состояние - перестановка с обменом местами двух случайно выбранных символов;
* $p(x')$ - произведение вероятностей биграмм (вероятности посчитаны на основе корпуса как количество биграмм, деленное на общее число биграмм) в тексте, расшифрованном с помощью новой перестановки;
* $p(x_t)$ - произведение вероятностей в тексте, расшифрованном с помощью текущей перестановки;
* Вероятность принятия новой перестановки: $P = \frac{p(x')}{p(x_t)} = e^{log(p(x')-log(p(x_t))}$

Процесс запускается несколько раз для определения состояния с наибольшим логарифмом праводоподобия ($log(P))$.

* **реализуйте и протестируйте его, убедитесь, что результаты улучшились**

In [44]:
def calc_log_likelihood(decoded_text, train_freqs):
    total_bigrams = np.array(list(train_freqs.values())).sum()
    log_likelihood = 0
    for i in range(len(decoded_text) - 1):
        bigram = decoded_text[i] + decoded_text[i+1]
        log_likelihood += np.log(train_freqs[bigram] / total_bigrams + 1e-4)
    return log_likelihood

In [46]:
final_decoded = None
max_likelihood = None

for j in range(5):
    cur_alphabet = alphabet.copy()

    for i in range(50000):
        test_text_decoded = change_alphabet(cur_alphabet, alphabet, test_text_encoded)

        if i % 10000 == 0:
            print(f'Текст после итерации {i}: {test_text_decoded}')

        cur_likelihood = calc_log_likelihood(test_text_decoded, train_bigram_freqs)

        next_alphabet = cur_alphabet.copy()
        sym1_idx = np.random.randint(len(next_alphabet))
        sym2_idx = np.random.randint(len(next_alphabet)) 
        while sym2_idx == sym1_idx:
            sym2_idx = np.random.randint(len(next_alphabet))
        next_alphabet[sym1_idx], next_alphabet[sym2_idx] = next_alphabet[sym2_idx], next_alphabet[sym1_idx]

        test_text_decoded = change_alphabet(next_alphabet, alphabet, test_text_encoded)
        next_likelihood = calc_log_likelihood(test_text_decoded, train_bigram_freqs)

        accept = False
        if next_likelihood > cur_likelihood:
            accept = True
        else:
            accept = np.random.rand() < (np.exp(next_likelihood - cur_likelihood))

        if accept:
            cur_alphabet = next_alphabet
    
    test_text_decoded = change_alphabet(cur_alphabet, alphabet, test_text_encoded)
    fin_likelihood = calc_log_likelihood(test_text_decoded, train_bigram_freqs) 
    if max_likelihood is None:
        max_likelihood = fin_likelihood
        final_decoded = test_text_decoded
    elif fin_likelihood >= max_likelihood:
        max_likelihood = fin_likelihood
        final_decoded = test_text_decoded
    print(f"Log likelihood {j}: {fin_likelihood}", end='\n\n')

Текст после итерации 0: хвряъщяпеалъряюлшчйеътлщлъткв впъяхъняшвъщсъмхлъ хвйеъыйяълтпдъквкяэъхдщуреъгвкйъдрлйъюшв шл ътърпдххяэъолнефъпяидылткдцъ вкпфылхдэъ хвыдйълияъмябхяъдтйяпкяювйеъдхвылъюъкяшящклъплбвпяърюлъндпфпдъюъярхяэътярлшбвптчътмлшйлпехсэъчрършуивчътяюлшалххяъщл юшлрхвчъквкъжйяъчъхлъряиврвптчъшвхеалъылмъуюдрлпъкяшящку
Текст после итерации 10000: наве делыхо вегоряты содо сказал ен бера дь пно знаты мте осли какей нидувы факт ивот гразроз с влинней чобыю лешимоскиц заклюмоний знамит оше пежне истелкегаты инамо г кередко ложале вго билюли г евней севоржался спортолыньй яв врушая сегорхонне дозгровная как эте я но вешавался раныхо моп угивол кередку
Текст после итерации 20000: наде бельхо дегорять собо свазал ен пера бы мно знать кте осли вавей нибудь чавт идот гразроз с длинней щопью лежикосвиц завлюконий знакит оже мешне истелвегать инако г веребво лошале дго пилюли г едней седоршался смортольный яд дружая сегорхонне бозгродная вав эте я но дежадался раньхо ком угидол веребву

In [47]:
final_decoded

'наво больше водерять себе сказал он пора бы мне знать что если какой нибувь факт ивет дразрез с влинной цепью логических заключений значит его можно истолкодать иначе д коробке лежало вде пилюли д овной совержался смертельный яв вругая содершенно бездревная как это я не вогавался раньше чем удивел коробку'

In [48]:
print(f"Доля правильно расшифрованных букв: {round(calc_accuracy(test_text, final_decoded), 2)}%")

Доля правильно расшифрованных букв: 92.48%


**4. Расшифруйте сообщение:**
**←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏**


Аналогично п.3, но добавлены 5 символов к алфавиту зашифрованного сообщения, чтобы выравнять количество символов в алфавитах

In [40]:
encoded_message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [43]:
final_decoded = None
max_likelihood = None

for j in range(5):
    cur_alphabet = list(set(encoded_message)) + ['!', '@', '#', '$', '%']

    for i in range(50000):
        test_text_decoded = change_alphabet(cur_alphabet, alphabet, encoded_message)

        if i % 10000 == 0:
            print(f'Текст после итерации {i}: {test_text_decoded}')

        cur_likelihood = calc_log_likelihood(test_text_decoded, train_bigram_freqs)

        next_alphabet = cur_alphabet.copy()
        sym1_idx = np.random.randint(len(next_alphabet))
        sym2_idx = np.random.randint(len(next_alphabet)) 
        while sym2_idx == sym1_idx:
            sym2_idx = np.random.randint(len(next_alphabet))
        next_alphabet[sym1_idx], next_alphabet[sym2_idx] = next_alphabet[sym2_idx], next_alphabet[sym1_idx]

        test_text_decoded = change_alphabet(next_alphabet, alphabet, encoded_message)
        next_likelihood = calc_log_likelihood(test_text_decoded, train_bigram_freqs)

        accept = False
        if next_likelihood > cur_likelihood:
            accept = True
        else:
            accept = np.random.rand() < (np.exp(next_likelihood - cur_likelihood))

        if accept:
            cur_alphabet = next_alphabet
    
    test_text_decoded = change_alphabet(cur_alphabet, alphabet, encoded_message)
    fin_likelihood = calc_log_likelihood(test_text_decoded, train_bigram_freqs) 
    if max_likelihood is None:
        max_likelihood = fin_likelihood
        final_decoded = test_text_decoded
    elif fin_likelihood >= max_likelihood:
        max_likelihood = fin_likelihood
        final_decoded = test_text_decoded
    print(f"Log likelihood {j}: {fin_likelihood}", end='\n\n')

Текст после итерации 0: схпкбйгбйкикцсбмчъовптмглбкпкбрчдцкбмчъовптмглбцснхцбшбецч чбхччжфсмкзбнчцчъглбпс нчбръчдкцвцтбхнчъссбйхс чбйгбйхсбхиспвпкбръвйкптмчбкбрчпшдкцсбовнхковптмглбжвппбщвбрчхпсимссбдсцйсъцчсбщвивмксбншъхвбачцзбнчмсдмчбзбмкдс чбмсбчжсфву
Текст после итерации 10000: если вы вимите норзальный или подти норзальный текст у этого соочшения который легко продитать скорее всего вы все смелали правильно и полудите заксизальный чалл ба послемнее детвертое бамание курса хотя конедно я нидего не очешащ
Текст после итерации 20000: если вы вимите норзальный или почти норзальный текст у этого соодцения который легко прочитать скорее всего вы все смелали правильно и получите заксизальный далл жа послемнее четвертое жамание курса ботя конечно я ничего не одецаш
Текст после итерации 30000: если вы визите нордальный или помти нордальный текст у чтого сообжения который легко промитать скорее всего вы все сзелали правильно и полумите даксидальный балл фа послезнее метвертое фазание курса 

In [ ]:
final_decoded

'если вы вимите норзальный или подти норзальный текст у этого сообшения который легко продитать скорее всего вы все смелали правильно и полудите заксизальный балл ча послемнее детвертое чамание курса цотя конедно я нидего не обешаю'

если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю